<a href="https://colab.research.google.com/github/HarshAhalpara-001/VoiceFlow/blob/main/NPL_Voice_Assistance_Model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! python --version

Python 3.11.11


In [ ]:
import random
import json

# Define phrases and entities for each intent
intent_data = {
    "copy_text": {
        "phrases": [
            "Copy this text",
            "Copy the selected text",
            "Copy the following text",
            "Copy this to the clipboard",
            "Copy the text to clipboard",
            "Copy that",
            "Copy the highlighted text",
            "Copy the content",
            "Copy the paragraph",
            "Copy the sentence",
            "Copy everything",
            "Copy the last line",
            "Copy the first paragraph",
            "Copy the selected content",
            "Copy the text from here"
        ],
        "entities": []
    },
    "paste_text": {
        "phrases": [
            "Paste the text",
            "Paste the copied text",
            "Paste from clipboard",
            "Paste the text here",
            "Paste the clipboard content",
            "Paste that",
            "Paste the content",
            "Paste the paragraph",
            "Paste the sentence",
            "Paste everything",
            "Paste the last line",
            "Paste the first paragraph",
            "Paste the selected content",
            "Paste the text from clipboard",
            "Paste here"
        ],
        "entities": []
    },
    "shutdown_system": {
        "phrases": [
            "Shutdown the system",
            "Shutdown the computer",
            "Turn off the system",
            "Power off the computer",
            "Shutdown now",
            "Shutdown the PC",
            "Shutdown the machine",
            "Turn off the PC",
            "Power down the system",
            "Shutdown immediately",
            "Shutdown the device",
            "Shutdown the laptop",
            "Shutdown the desktop",
            "Shutdown the server",
            "Shutdown the workstation"
        ],
        "entities": []
    },
    "restart_system": {
        "phrases": [
            "Restart the system",
            "Restart the computer",
            "Reboot the system",
            "Restart now",
            "Reboot the computer",
            "Restart the PC",
            "Restart the machine",
            "Reboot the PC",
            "Restart immediately",
            "Reboot the device",
            "Restart the laptop",
            "Restart the desktop",
            "Restart the server",
            "Restart the workstation",
            "Reboot now"
        ],
        "entities": []
    },
    "open_app": {
        "phrases": [
            "Open {app}",
            "Launch {app}",
            "Start {app}",
            "Run {app}",
            "Open the {app} application",
            "Open {app} for me",
            "Launch the {app} app",
            "Start the {app} program",
            "Run the {app} software",
            "Open {app} now",
            "Open {app} quickly",
            "Open {app} immediately",
            "Open {app} on my computer",
            "Open {app} on this machine",
            "Open {app} on the system"
        ],
        "entities": {
            "app": ["Chrome", "Firefox", "Word", "Excel", "Notepad", "PowerPoint", "Photoshop", "Edge", "Outlook", "Teams", "Zoom", "Slack", "VSCode", "Sublime", "Calculator"]
        }
    },
    "adjust_volume": {
        "phrases": [
            "Increase the volume to {level}",
            "Set the volume to {level}",
            "Decrease the volume to {level}",
            "Change the volume to {level}",
            "Adjust the volume to {level}",
            "Turn up the volume to {level}",
            "Turn down the volume to {level}",
            "Set the sound to {level}",
            "Increase the sound to {level}",
            "Decrease the sound to {level}",
            "Change the sound to {level}",
            "Adjust the sound to {level}",
            "Set the audio to {level}",
            "Increase the audio to {level}",
            "Decrease the audio to {level}"
        ],
        "entities": {
            "level": ["50 percent", "100 percent", "25 percent", "75 percent", "10 percent", "20 percent", "30 percent", "40 percent", "60 percent", "70 percent", "80 percent", "90 percent", "max", "min", "medium"]
        }
    },
    "terminate_app": {
        "phrases": [
            "Terminate {app}",
            "Close {app}",
            "Kill {app}",
            "End {app}",
            "Stop {app}",
            "Quit {app}",
            "Exit {app}",
            "Force close {app}",
            "Shut down {app}",
            "Terminate the {app} process",
            "Close the {app} application",
            "Kill the {app} program",
            "End the {app} task",
            "Stop the {app} software",
            "Quit the {app} app"
        ],
        "entities": {
            "app": ["Chrome", "Firefox", "Word", "Excel", "Notepad", "PowerPoint", "Photoshop", "Edge", "Outlook", "Teams", "Zoom", "Slack", "VSCode", "Sublime", "Calculator"]
        }
    }
}

# Function to generate synthetic data
def generate_synthetic_data(intent_data, num_examples=1000):
    synthetic_data = []
    intents = list(intent_data.keys())
    examples_per_intent = num_examples // len(intents)

    for intent in intents:
        phrases = intent_data[intent]["phrases"]
        entities = intent_data[intent]["entities"]

        for _ in range(examples_per_intent):
            phrase = random.choice(phrases)
            entity_list = []

            # Replace placeholders with entity values
            if entities:
                for entity_type, entity_values in entities.items():
                    entity_value = random.choice(entity_values)
                    start = phrase.find("{" + entity_type + "}")
                    end = start + len(entity_value)
                    phrase = phrase.replace("{" + entity_type + "}", entity_value)
                    entity_list.append((start, end, entity_type.upper()))

            synthetic_data.append({
                "text": phrase,
                "intent": intent,
                "entities": entity_list
            })

    return synthetic_data

# Generate 300 examples
synthetic_data = generate_synthetic_data(intent_data, num_examples=1000)

# Save the synthetic data to a JSON file
with open("synthetic_training_data.json", "w") as f:
    json.dump(synthetic_data, f, indent=2)

print("Synthetic training data generated and saved to 'synthetic_training_data.json'.")


Synthetic training data generated and saved to 'synthetic_training_data.json'.


In [ ]:
import spacy
from spacy.training import Example
from spacy.util import minibatch, compounding
import random
import json
import os
import pyperclip

# Step 1: Create Training Data with Entities
training_data = [
    {"text": "Copy this text", "intent": "copy_text", "entities": []},
    {"text": "Paste the text", "intent": "paste_text", "entities": []},
    {"text": "Shutdown the system", "intent": "shutdown_system", "entities": []},
    {"text": "Restart the computer", "intent": "restart_system", "entities": []},
    {"text": "Open Chrome", "intent": "open_app", "entities": [(5, 10, "APP_NAME")]},  # "Chrome" is an entity
    {"text": "Increase the volume to 50 percent", "intent": "adjust_volume", "entities": [(22, 32, "VOLUME_LEVEL")]},  # "50 percent" is an entity
    {"text": "Terminate Chrome", "intent": "terminate_app", "entities": [(10, 15, "APP_NAME")]}  # "Chrome" is an entity
]
training_data = synthetic_data
# Save training data to a JSON file
with open("training_data.json", "w") as f:
    json.dump(training_data, f)

# Step 2: Load or Create a Model
nlp = spacy.blank("en")  # Start with a blank model

# Add a text classification pipeline
if "textcat" not in nlp.pipe_names:
    textcat = nlp.add_pipe("textcat")
else:
    textcat = nlp.get_pipe("textcat")

# Add labels for new intents
new_intents = ["copy_text", "paste_text", "shutdown_system", "restart_system", "open_app", "adjust_volume", "terminate_app"]
for intent in new_intents:
    textcat.add_label(intent)

# Add a NER pipeline for entity recognition
if "ner" not in nlp.pipe_names:
    ner = nlp.add_pipe("ner")
else:
    ner = nlp.get_pipe("ner")

# Add entity labels
ner.add_label("APP_NAME")
ner.add_label("VOLUME_LEVEL")

# Step 3: Convert Training Data to spaCy Format
def create_training_data(training_data):
    train_data = []
    for item in training_data:
        text = item["text"]
        intent = item["intent"]
        entities = item["entities"]
        train_data.append((text, {"cats": {intent: 1.0}, "entities": entities}))
    return train_data

train_data = create_training_data(training_data)

# Step 4: Train the Model
random.seed(1)
spacy.util.fix_random_seed(1)

n_iter = 25  # Number of training iterations
optimizer = nlp.begin_training()

# Training loop
for i in range(n_iter):
    losses = {}
    random.shuffle(train_data)
    batches = minibatch(train_data, size=compounding(4.0, 32.0, 1.001))
    for batch in batches:
        texts, annotations = zip(*batch)
        examples = [Example.from_dict(nlp.make_doc(text), annotation) for text, annotation in zip(texts, annotations)]
        nlp.update(examples, drop=0.2, losses=losses, sgd=optimizer)
    print(f"Losses at iteration {i}: {losses}")

# Save the updated model
nlp.to_disk("winTA_model_updated")
print("Model training complete and saved as 'winTA_model_updated'.")

# Step 5: Load the Updated Model
nlp = spacy.load("winTA_model_updated")

# Step 6: Function to Predict Intent and Entities
def predict_intent_and_entities(text):
    doc = nlp(text)
    intent = max(doc.cats, key=doc.cats.get)
    confidence = doc.cats[intent]
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    print(f"User Input: {text}\nPredicted Intent: {intent} (Confidence: {confidence:.2f})\nEntities: {entities}\n")
    return intent, entities

# Step 7: Function to Handle Intents and Entities
def handle_intent(intent, entities, text):
    if intent == "copy_text":
        # pyperclip.copy(text)
        print("Text copied to clipboard.")
    elif intent == "paste_text":
        # pasted_text = pyperclip.paste()
        print(f"Pasted Text: intent")
    elif intent == "shutdown_system":
        # os.system("shutdown /s /t 1")  # Windows shutdown command
        print("System is shutting down...")
    elif intent == "restart_system":
        # os.system("shutdown /r /t 1")  # Windows restart command
        print("System is restarting...")
    elif intent == "open_app":
        app_name = next((ent[0] for ent in entities if ent[1] == "APP_NAME"), None)
        if app_name:
            # os.system(f"start {app_name}")  # Open app on Windows
            print(f"Opening {app_name}...")
        else:
            print("App name not found.")
    elif intent == "adjust_volume":
        volume_level = next((ent[0] for ent in entities if ent[1] == "VOLUME_LEVEL"), None)
        if volume_level:
            print(f"Adjusting volume to {volume_level}...")  # Add volume control logic here
        else:
            print("Volume level not found.")
    elif intent == "terminate_app":
        app_name = next((ent[0] for ent in entities if ent[1] == "APP_NAME"), None)
        if app_name:
            # os.system(f"taskkill /im {app_name}.exe /f")  # Terminate app on Windows
            print(f"Terminating {app_name}...")
        else:
            print("App name not found.")
    else:
        print("Command not recognized.")

# Step 8: Test Cases
test_cases = [
    "Copy this text",
    "Paste the text",
    "Shutdown the system",
    "Restart the computer",
    "Open Chrome",
    "Increase the volume to 50 percent",
    "Terminate Chrome"
]

for text in test_cases:
    intent, entities = predict_intent_and_entities(text)
    handle_intent(intent, entities, text)

Losses at iteration 0: {'textcat': 6.76493893942639, 'ner': 254.38085099593434}
Losses at iteration 1: {'textcat': 0.009070839977528389, 'ner': 0.7712125528469187}
Losses at iteration 2: {'textcat': 4.6479108645865186e-05, 'ner': 3.18700024635531e-05}
Losses at iteration 3: {'textcat': 0.00015824144772468018, 'ner': 9.468519684864647e-06}
Losses at iteration 4: {'textcat': 4.274889506442753e-06, 'ner': 2.0038451817808857e-06}
Losses at iteration 5: {'textcat': 2.1438838033402147e-06, 'ner': 1.8684966713094286}
Losses at iteration 6: {'textcat': 1.0031805326951582e-05, 'ner': 15.489717479323726}
Losses at iteration 7: {'textcat': 5.080716677205617e-07, 'ner': 1.5855568613682478}
Losses at iteration 8: {'textcat': 2.3738956065116e-06, 'ner': 4.1674389341147045e-07}
Losses at iteration 9: {'textcat': 8.359431524554062e-05, 'ner': 0.0005450278793909682}
Losses at iteration 10: {'textcat': 5.7636302021375725e-05, 'ner': 6.369199304763343}
Losses at iteration 11: {'textcat': 2.96658439267235

In [ ]:
from google.colab import files
import shutil

# Zip the folder
shutil.make_archive('/content/winTA_model_updated', 'zip', '/content/winTA_model_updated')

# Download the zipped folder
files.download('/content/winTA_model_updated.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>